In [2]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *

In [3]:
df = unpkl('raw.pkl')

# Enumeration of Cases

**<font color='red'>NOTE: I have not processed the new texture captures that Joitree took. I'll have to do that.</font>**

In [13]:
for catalog in df.catalog.unique():
    tmp = df[df.catalog==catalog]
    tmp.mtrial = tmp.mtrial.apply(int)
    tmp = tmp.sort_values('mtrial')
    
    # if all measurements are from same dir, have same manbran, and trial indices are consecutive, then ignore
    if all([len(tmp.samploc.unique())==1,len(tmp.manbran.unique())==1,is_consecutive(tmp.mtrial)]):
        continue
    else:
        print(catalog,end='\t')

5149	5150	5151	5154	97	1963	82	4213	5198	2198	4231	4945	4949	4783s	1882c	289c	289d	289e	289f	2073z	3202a	